## Import Libraries

In [71]:
import numpy as np
import pandas as pd
import gensim
import logging
import re
import nltk.data
from nltk.corpus import stopwords
import pysentiment as ps
from nltk.stem.porter import *
stemmer = PorterStemmer() 
from __future__ import division
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import codecs
import nltk.tag.stanford as st
tagger = st.StanfordNERTagger('stanford-ner-2014-06-16/stanford-ner-2014-06-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                             'stanford-ner-2014-06-16/stanford-ner-2014-06-16/stanford-ner.jar')

from nltk.stem.porter import *

In [2]:
stemmer = PorterStemmer()

In [3]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [4]:
stops = set(stopwords.words("english"))


##  Reading Lougran Dictionary

In [5]:
logran_dict = pd.read_csv('lougran.csv',sep=',')

In [6]:
pos_lougran = []
neg_lougran = []
for index,row in logran_dict.iterrows():
    if row['Positive'] == 2009:
        pos_lougran.append(row['Word'].lower())
    if row['Negative'] == 2009:
        neg_lougran.append(row['Word'].lower()) 

In [7]:
len(pos_lougran)

352

In [8]:
len(neg_lougran)

2315

In [9]:
def lex_Lougran():
    words_neg_lougran = []
    fname = 'Loughran_McDonald_AggregateIPOWordList.txt'
    f = open(fname)
    for l in f.readlines():
       
        word = l.strip().lower()
        words_neg_lougran.append(word)
       
    return words_neg_lougran


In [10]:
words_neg_lougran = lex_Lougran()

In [11]:
len(words_neg_lougran)

2600

## Reading Dataset


In [5]:
training_data = pd.read_json(codecs.open('Headline_Trainingdata.json', 'r', 'utf-8'),orient='records')
test_data = pd.read_json(codecs.open('Headline_Testingdata.json', 'r', 'utf-8'),orient='records')


In [14]:
test_data.shape

(491, 6)

In [15]:
training_data.shape

(1142, 7)

In [16]:
def lougran_pos_count(text):
    words = text.split()
    pos_count = 0
    for w in words:
        if w.lower() in pos_lougran:
            pos_count = pos_count+1
    return pos_count

def lougran_neg_count(text):
    words = text.split()
    neg_count = 0
    for w in words:
        if w.lower() in words_neg_lougran:
            neg_count = neg_count+1
    return neg_count

In [17]:
# Computing Lougran Sentiment Scores
training_data['neg_count'] = training_data['title'].apply(lougran_neg_count)
training_data['pos_count'] = training_data['title'].apply(lougran_pos_count)

In [20]:
test_data['neg_count'] = test_data['title'].apply(lougran_neg_count)
test_data['pos_count'] = test_data['title'].apply(lougran_pos_count)

In [12]:
words_lex = []
score_lex = []
def lex_SCL():
    fname = 'SCL-NMA.txt'
    f = open(fname)
    for l in f.readlines():
       
        word,score = l.split('\t')
        words_lex.append(word)
        score = score.replace('\n','')
        score_lex.append(float(score))
    return words_lex,score_lex


In [13]:
words_lex_pmi = []
score_lex_pmi = []
def lex_PMI():
    fname = 'unigrams-pmilexicon.txt'
    f = open(fname)
    for l in f.readlines():
       
        word,score,pos,neg = l.split('\t')
        #print word
        #print word.startswith('@')
        if(word.startswith('@') == False and (word[0:3] != 'http')):
            words_lex_pmi.append(word)
            #score = score.replace('\n','')
            score_lex_pmi.append(float(score))
      
    return words_lex_pmi,score_lex_pmi


In [14]:
words_lex,score_lex = lex_SCL()
words_lex_pmi,score_lex_pmi = lex_PMI()

In [15]:
len(words_lex_pmi)

44668

In [16]:
words_lex_pmi[0:10]

['familar',
 'emilio',
 'on-site',
 'supertramp',
 'spinvox',
 'studmuffin',
 'bessst',
 'rowling',
 'winplace.at',
 'wahooooooo']

## Preprocessing Text

In [21]:
# not required for spans
def cleanText(text,companyName):
    text = re.sub("[^a-zA-Z]"," ", text)
    tagged = tagger.tag(text.split())
    company = companyName.split(' ')

    newText = ''
    for word in tagged:
        if word[1] in ['ORGANIZATION','PERSON']:
          
            newText = newText +  ' '+ word[0].upper()
        elif word[0] in company or word[0].isupper():
            
            newText = newText +  ' '+ word[0].upper()
        else:
           
            newText = newText + ' ' + word[0].lower()
            
  
   
    words = newText.split()
    words = [w for w in words if len(w) > 1]
    return ( " ".join(words) )
   

In [35]:
cleanTextArray = []
for index,row in training_data.iterrows():
    print index
    cleanTextArray.append(cleanText(row['title'],row['company']))
training_data['cleanText'] = cleanTextArray

In [38]:
cleanTextArray = []
for index,row in test_data.iterrows():
    print index
    cleanTextArray.append(cleanText(row['title'],row['company']))
test_data['cleanText'] = cleanTextArray

In [40]:
def lex_score(text):
    #tokens = lm.tokenize(text)
    #score = lm.get_score(tokens)
    tokens = text.split(' ')
    score = 0
    for word in tokens:
        for index in range(0,len(words_lex)):
            if(word.isupper() == False):
                if stemmer.stem(str(word.lower())) == stemmer.stem(words_lex[index]):
                        score = score+score_lex[index]
                        #print score
                        #print word
                        break

    return score

In [261]:
def lex_score_pmi(text):
    #tokens = lm.tokenize(text)
    #score = lm.get_score(tokens)
    text = re.sub("[^a-zA-Z-]"," ", text)
    #text = re.sub('-',' ',text)
    #text = re.sub("'",'',text)
    words = nltk.word_tokenize(text)
    tags = nltk.pos_tag(words)
    print tags
    #tokens = text.split(' ')
    score = 0
    print tags
    for word in tags:
        if word[1] in ['VBD','JJR','JJ','VB','VBS','VBZ','VBN','VBG','VBP','RB','RP']:
            print word[0]
            for index in range(0,len(words_lex_pmi)):
                if(word[0].isupper() == False):
                    if stemmer.stem(word[0]) == stemmer.stem(words_lex_pmi[index].decode('utf-8')):
                            score = score+score_lex_pmi[index]
                            print words_lex_pmi[index]
                            print score_lex_pmi[index]

                            break

    return score

In [249]:
#lm = ps.LM()
hiv4 = ps.HIV4()
def lex_score2(text):
    #tokens = lm.tokenize(text)
    #score = lm.get_score(tokens)
    tokens = hiv4.tokenize(text)  # text can be tokenized by other ways
                                  # however, dict in HIV4 is preprocessed
                                  # by the default tokenizer in the library
    score = hiv4.get_score(tokens)

    return score['Polarity']

In [208]:
training_data['lex_score3'] = training_data['cleanText'].apply(lex_score_pmi)

In [209]:
training_data['lex_score2'] = training_data['cleanText'].apply(lex_score2)

In [210]:
test_data['lex_score3'] = test_data['cleanText'].apply(lex_score_pmi)
test_data['lex_score2'] = test_data['cleanText'].apply(lex_score2)

In [1]:
## Train and test data preparation for training

In [283]:
X_train = training_data[['id','title','cleanText','lex_score3','lex_score2','pos_count','neg_count']]
X_test = test_data[['id','title','cleanText','lex_score3','lex_score2','pos_count','neg_count']]
y_train = training_data[['sentiment']]


In [284]:
from sklearn.metrics import make_scorer,r2_score
from sklearn.metrics.pairwise import cosine_similarity

In [285]:
def customerscorer(y,y_pred):
    return cosine_similarity([y],[y_pred])

In [286]:
cs_scorer = make_scorer(r2_score)

In [288]:
# . TF-IDF Features
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
X_training = vectorizer.fit_transform(X_train.title)
X_testing = vectorizer.transform(X_test.title)

In [289]:
# Unigram Features
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1), max_df=0.5,
                                 stop_words='english')
X_trainingGram = vectorizer.fit_transform(X_train.title)
X_testingGram = vectorizer.transform(X_test.title)

In [290]:
# Parameter tuning
x_params = { 'max_depth':range(3,10,2),
             'n_estimators': [100,150],
             'min_child_weight' :range(1,6,2),
             'objective':['reg:linear'],
             'scale_pos_weight' : [1,5,10]
       }
xgb_model = xgb.XGBRegressor()

In [291]:
X_training.shape

(1142, 2503)

In [292]:
X_testing.shape

(491, 2503)

In [293]:
# Model 1- Feature Matrix
train_feats =np.column_stack((X_training.toarray(),X_train['pos_count'],X_train['neg_count'],X_train['lex_score3'],X_train['lex_score2']))
test_feats =np.column_stack((X_testing.toarray(),X_test['pos_count'],X_test['neg_count'],X_test['lex_score3'],X_test['lex_score2']))

# Model 2- Feature Matrix
train_feats1 =np.column_stack((X_trainingGram.toarray(),X_train['pos_count'],X_train['neg_count'],X_train['lex_score3'],X_train['lex_score2']))
test_feats1 =np.column_stack((X_testingGram.toarray(),X_test['pos_count'],X_test['neg_count'],X_test['lex_score3'],X_test['lex_score2']))


## Model 1- Training

In [295]:
clf = xgb.XGBRegressor(n_estimators=500,seed = 23)
clf.fit(train_feats1,y_train.sentiment)


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=23, silent=True, subsample=1)

## Model2- Training

In [296]:
clf_tfidf = xgb.XGBRegressor(n_estimators=100,seed = 23)
clf_tfidf.fit(train_feats,y_train.sentiment)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=23, silent=True, subsample=1)

In [297]:
#Predictions from both models
pr = clf.predict(test_feats1)
pr_tfidf = clf_tfidf.predict(test_feats)

In [299]:
## Computing Mean predictions
y_test = X_test[['id']]
y_test['predicted sentiment'] = np.mean((pr,pr_tfidf),axis=0)
y_train['predicted sentiment'] = np.mean((clf.predict(train_feats1),clf_tfidf.predict(train_feats)),axis=0)

In [304]:
submission = pd.DataFrame({'id':X_test['id'],'sentiment score' : y_test['predicted sentiment']})

In [305]:
submission.head()

,id,sentiment score
0,1144,0.395600
1,1145,0.272562
2,1146,-0.395455
3,1147,0.170142
4,1148,-0.438649


In [86]:
submission.head()

,id,sentiment score
0,1144,0.691899
1,1145,0.191305
2,1146,-0.373309
3,1147,0.083100
4,1148,-0.658537


In [306]:
#Writing submission file
submission.to_json('submission.json',orient='records')